## Creating Chat Assistant Using Llama index and LangChain

In [1]:
import nest_asyncio
nest_asyncio.apply()

### OpenAI API Key And GitHub Token

In [ ]:
%env OPENAI_API_KEY=sk-*******************************
from llama_index import GPTVectorStoreIndex, GithubRepositoryReader
from IPython.display import Markdown, display
import os

In [ ]:
%env GITHUB_TOKEN=ghp_*********************************
github_token = os.environ.get("GITHUB_TOKEN")
owner = "dair-ai"
repo = "Prompt-Engineering-Guide"
branch = "main"

documents = GithubRepositoryReader(
    github_token=github_token,
    owner=owner,
    repo=repo,
    use_parser=False,
    verbose=False,
).load_data(branch=branch)

In [71]:
from llama_index import download_loader, GPTVectorStoreIndex, ServiceContext, StorageContext
service_context = ServiceContext.from_defaults(chunk_size_limit=512)

storage_context = StorageContext.from_defaults()
index = GPTVectorStoreIndex.from_documents(
        documents, 
        service_context=service_context,
        storage_context=storage_context,
    )
storage_context.persist(persist_dir=f'./storage/index')

In [5]:
from llama_index import  load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir=f'./storage/index')
index = load_index_from_storage(storage_context=storage_context)

### Creating Composable Graph Index

In [6]:
from llama_index import GPTListIndex, LLMPredictor, ServiceContext, load_graph_from_storage
from langchain import OpenAI
from llama_index.indices.composability import ComposableGraph

index_summary = f"OpenAI Cookbook repository"

# define an LLMPredictor set number of output tokens
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, max_tokens=512))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
storage_context = StorageContext.from_defaults()

# define a list index over the vector indices
# allows us to synthesize information across each index
graph = ComposableGraph.from_indices(
    GPTListIndex,
    [index], 
    index_summaries=[index_summary],
    service_context=service_context,
    storage_context = storage_context,
)
root_id = graph.root_id

storage_context.persist(persist_dir=f'./storage/root')

In [7]:
graph = load_graph_from_storage(
    root_id=root_id, 
    service_context=service_context,
    storage_context=storage_context,
)

### Building Query Engine Using LangChain Chat_models

In [8]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig

In [9]:
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
decompose_transform = DecomposeQueryTransform(
    llm_predictor, verbose=True
)

# define custom retrievers
from llama_index.query_engine.transform_query_engine import TransformQueryEngine

query_engine = index.as_query_engine()
query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )

In [10]:
graph_config = IndexToolConfig(
    query_engine=query_engine,
    name=f"Graph Index",
    description="useful for when you want to answer queries that require analyzing repo.",
    tool_kwargs={"return_direct": True}
)

In [11]:
tool_config = IndexToolConfig(
        query_engine=query_engine, 
        name=f"Vector Index",
        description=f"useful for when you want to answer queries about the repo",
        tool_kwargs={"return_direct": True}
    )

In [12]:
toolkit = LlamaToolkit(
    index_configs= [tool_config] + [graph_config],
)

### Conversational Chat Agent

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
    verbose=True
)

In [88]:
agent_chain.run(input="hi, i am ram")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Ram, nice to meet you! How can I help you today?

> Finished chain.


'Hi Ram, nice to meet you! How can I help you today?'

In [54]:
agent_chain.run(input="I'm trying to learn about chatgpt and recently worked with text davinci 002 model.")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: ChatGPT is an open-source natural language processing (NLP) model developed by OpenAI. It is based on the GPT-2 model and is designed to generate human-like text. Text-Davinci 002 is a text-generating model developed by OpenAI. It is based on the GPT-2 model and is designed to generate human-like text. Both models are powerful tools for natural language processing and can be used to generate text, answer questions, and provide insights into a wide range of topics.

> Finished chain.


'ChatGPT is an open-source natural language processing (NLP) model developed by OpenAI. It is based on the GPT-2 model and is designed to generate human-like text. Text-Davinci 002 is a text-generating model developed by OpenAI. It is based on the GPT-2 model and is designed to generate human-like text. Both models are powerful tools for natural language processing and can be used to generate text, answer questions, and provide insights into a wide range of topics.'

In [92]:
while True:
    text_input = input("User: ")
    if text_input == 'Thank you':
        break
    response = agent_chain.run(input=text_input)
    print(f'Agent: {response}')

User: Hello!, this is ram.


> Entering new AgentExecutor chain...

AI: Hi Ram, nice to meet you! How can I help you today?

> Finished chain.
Agent: Hi Ram, nice to meet you! How can I help you today?
User: Can you tell me about AI Cookbook?


> Entering new AgentExecutor chain...

AI: The AI Cookbook is a collection of tutorials and examples for using AI technologies. It provides step-by-step instructions for building and training models, as well as tips and tricks for getting the most out of AI technologies. The Cookbook also includes a variety of sample code snippets and examples to help you get started quickly.

> Finished chain.
Agent: The AI Cookbook is a collection of tutorials and examples for using AI technologies. It provides step-by-step instructions for building and training models, as well as tips and tricks for getting the most out of AI technologies. The Cookbook also includes a variety of sample code snippets and examples to help you get started quickly.
User: Can you 

In [93]:
while True:
    text_input = input("User: ")
    if text_input == 'Thank you':
        break
    response = agent_chain.run(input=text_input)
    print(f'Agent: {response}')

User: Hi, this is sai.


> Entering new AgentExecutor chain...

AI: Hi Sai, nice to meet you! How can I help you today?

> Finished chain.
Agent: Hi Sai, nice to meet you! How can I help you today?
User: I'm working on building some chatGPT models.


> Entering new AgentExecutor chain...

AI: Great! What kind of chatGPT models are you building?

> Finished chain.
Agent: Great! What kind of chatGPT models are you building?
User: I started working on GPT Turbo 3.5 by giving it custom input data.


> Entering new AgentExecutor chain...

AI: Sounds interesting! What kind of custom input data are you giving GPT Turbo 3.5?

> Finished chain.
Agent: Sounds interesting! What kind of custom input data are you giving GPT Turbo 3.5?
User: I tried giving custom input in format of git repo, pdfs and excel.


> Entering new AgentExecutor chain...

AI: That's great! Using custom input data in the form of git repos, PDFs, and Excel files can help GPT Turbo 3.5 learn more effectively and provide more a